In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm_notebook
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns             

sns.set_style("darkgrid")
sys.path.append('/home/robot-i9/Desktop/Anand/')
print(sys.path)

from NeuralBlocks.models.densenet import DenseNet

['/home/robot-i9/Desktop/Anand/NeuralBlocks/notebooks', '/home/robot-i9/Desktop/Anand/NeuralBlocks/notebooks', '', '/home/robot-i9/Downloads/pynaoqi-python2.7-2.5.5.5-linux64/lib/python2.7/site-packages', '/home/robot-i9/miniconda3/envs/main/lib/python37.zip', '/home/robot-i9/miniconda3/envs/main/lib/python3.7', '/home/robot-i9/miniconda3/envs/main/lib/python3.7/lib-dynload', '/home/robot-i9/miniconda3/envs/main/lib/python3.7/site-packages', '/home/robot-i9/miniconda3/envs/main/lib/python3.7/site-packages/IPython/extensions', '/home/robot-i9/.ipython', '/home/robot-i9/Desktop/Anand/']


In [2]:
torch.manual_seed(2456)
cudnn.deterministic = True
cudnn.benchmark = False
np.random.seed(2456)

NUM_EPOCH = 200
BATCH_SIZE = 128
CHECKPOINT_INTERVAL = 100
LRS = [0.0001, 0.001, 0.1]
NORMS =[None,'BN', 'SN', 'WN', 'MSN', 'MSNTReLU', 'WNTReLU']
DATA_PATH = "/home/robot-i9/Desktop/Anand/NeuralBlocks/data_utils/datasets/CIFAR10/"
SAVE_PATH = "/home/robot-i9/Desktop/Anand/NeuralBlocks/experiments/CIFAR10/"

os.makedirs(SAVE_PATH, exist_ok=True)

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root=DATA_PATH, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root=DATA_PATH, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        train_loss_log.append(train_loss/(batch_idx+1))
        train_acc_log.append( 100.*correct/total)
        
        if(batch_idx%CHECKPOINT_INTERVAL==0):
             print("Train Epoch [{:3d}/{:3d}]Batch [{:3d}/{:3d}] Loss: {:.3f} Acc {:.3f}%".format(epoch, NUM_EPOCH,batch_idx, len(trainloader),
                train_loss/(batch_idx+1), 100.*correct/total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            test_loss_log.append(test_loss/(batch_idx+1))
            test_acc_log.append( 100.*correct/total)
        
            if(batch_idx%CHECKPOINT_INTERVAL==0):
                print("Test Epoch [{:3d}/{:3d}]Batch [{:3d}/{:3d}] Loss: {:.3f} Acc {:.3f}%".format(epoch, NUM_EPOCH,batch_idx, len(testloader),
                test_loss/(batch_idx+1), 100.*correct/total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir(SAVE_PATH+'checkpoint'):
            os.mkdir(SAVE_PATH+'checkpoint')
        torch.save(state, SAVE_PATH+'checkpoint/ckpt.pth')
        best_acc = acc

In [ ]:
best_acc = 0 

for lr in tqdm_notebook(LRS):
    for norm in tqdm_notebook(NORMS):
        net = DenseNet(3, 10, norm=norm).cuda()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(net.parameters(), lr=lr)

        train_loss_log =[]
        train_acc_log = []
        test_loss_log =[]
        test_acc_log =[]

        for epoch in tqdm_notebook(range(NUM_EPOCH)):
            train(epoch)
            test(epoch)

        np.save(SAVE_PATH+"DenseNet_Train_loss_{}_{}.npy".format(norm,lr), train_loss_log)  
        np.save(SAVE_PATH+"DenseNet_Test_loss_{}_{}.npy".format(norm,lr), test_loss_log)    
        np.save(SAVE_PATH+"DenseNet_Train_Acc_{}_{}.npy".format(norm,lr), train_acc_log)    
        np.save(SAVE_PATH+"DenseNet_Test_Acc_{}_{}.npy".format(norm,lr), test_acc_log)   
        del net

Train Epoch [  0/200]Batch [  0/391] Loss: 2.305 Acc 10.156%
Train Epoch [  0/200]Batch [100/391] Loss: 2.172 Acc 21.488%
Train Epoch [  0/200]Batch [200/391] Loss: 2.082 Acc 25.606%
Train Epoch [  0/200]Batch [300/391] Loss: 2.024 Acc 27.871%
Test Epoch [  0/200]Batch [  0/ 79] Loss: 1.747 Acc 43.750%
Saving..
Train Epoch [  1/200]Batch [  0/391] Loss: 1.790 Acc 40.625%
Train Epoch [  1/200]Batch [100/391] Loss: 1.797 Acc 36.231%
Train Epoch [  1/200]Batch [200/391] Loss: 1.765 Acc 37.725%
Train Epoch [  1/200]Batch [300/391] Loss: 1.745 Acc 38.312%
Test Epoch [  1/200]Batch [  0/ 79] Loss: 1.576 Acc 46.094%
Saving..
Train Epoch [  2/200]Batch [  0/391] Loss: 1.751 Acc 39.844%
Train Epoch [  2/200]Batch [100/391] Loss: 1.661 Acc 40.671%
Train Epoch [  2/200]Batch [200/391] Loss: 1.648 Acc 41.476%
Train Epoch [  2/200]Batch [300/391] Loss: 1.642 Acc 41.554%
Test Epoch [  2/200]Batch [  0/ 79] Loss: 1.513 Acc 50.000%
Saving..
Train Epoch [  3/200]Batch [  0/391] Loss: 1.588 Acc 46.094%


Train Epoch [ 26/200]Batch [200/391] Loss: 1.267 Acc 56.269%
Train Epoch [ 26/200]Batch [300/391] Loss: 1.263 Acc 56.263%
Test Epoch [ 26/200]Batch [  0/ 79] Loss: 1.131 Acc 60.938%
Saving..
Train Epoch [ 27/200]Batch [  0/391] Loss: 1.181 Acc 58.594%
Train Epoch [ 27/200]Batch [100/391] Loss: 1.256 Acc 56.660%
Train Epoch [ 27/200]Batch [200/391] Loss: 1.257 Acc 56.713%
Train Epoch [ 27/200]Batch [300/391] Loss: 1.256 Acc 56.494%
Test Epoch [ 27/200]Batch [  0/ 79] Loss: 1.135 Acc 61.719%
Train Epoch [ 28/200]Batch [  0/391] Loss: 1.341 Acc 50.000%
Train Epoch [ 28/200]Batch [100/391] Loss: 1.257 Acc 56.490%
Train Epoch [ 28/200]Batch [200/391] Loss: 1.252 Acc 56.627%
Train Epoch [ 28/200]Batch [300/391] Loss: 1.252 Acc 56.632%
Test Epoch [ 28/200]Batch [  0/ 79] Loss: 1.103 Acc 64.844%
Saving..
Train Epoch [ 29/200]Batch [  0/391] Loss: 1.147 Acc 62.500%
Train Epoch [ 29/200]Batch [100/391] Loss: 1.247 Acc 57.109%
Train Epoch [ 29/200]Batch [200/391] Loss: 1.250 Acc 56.860%
Train Epo

Saving..
Train Epoch [ 53/200]Batch [  0/391] Loss: 1.135 Acc 60.156%
Train Epoch [ 53/200]Batch [100/391] Loss: 1.163 Acc 59.793%
Train Epoch [ 53/200]Batch [200/391] Loss: 1.172 Acc 59.527%
Train Epoch [ 53/200]Batch [300/391] Loss: 1.172 Acc 59.346%
Test Epoch [ 53/200]Batch [  0/ 79] Loss: 1.026 Acc 66.406%
Train Epoch [ 54/200]Batch [  0/391] Loss: 1.163 Acc 56.250%
Train Epoch [ 54/200]Batch [100/391] Loss: 1.170 Acc 59.994%
Train Epoch [ 54/200]Batch [200/391] Loss: 1.167 Acc 59.857%
Train Epoch [ 54/200]Batch [300/391] Loss: 1.165 Acc 59.702%
Test Epoch [ 54/200]Batch [  0/ 79] Loss: 1.025 Acc 67.188%
Saving..
Train Epoch [ 55/200]Batch [  0/391] Loss: 1.270 Acc 52.344%
Train Epoch [ 55/200]Batch [100/391] Loss: 1.161 Acc 60.350%
Train Epoch [ 55/200]Batch [200/391] Loss: 1.168 Acc 59.534%
Train Epoch [ 55/200]Batch [300/391] Loss: 1.166 Acc 59.850%
Test Epoch [ 55/200]Batch [  0/ 79] Loss: 1.000 Acc 64.844%
Saving..
Train Epoch [ 56/200]Batch [  0/391] Loss: 0.982 Acc 75.781%


Test Epoch [ 79/200]Batch [  0/ 79] Loss: 0.960 Acc 66.406%
Train Epoch [ 80/200]Batch [  0/391] Loss: 1.133 Acc 57.812%
Train Epoch [ 80/200]Batch [100/391] Loss: 1.147 Acc 60.783%
Train Epoch [ 80/200]Batch [200/391] Loss: 1.131 Acc 61.140%
Train Epoch [ 80/200]Batch [300/391] Loss: 1.124 Acc 61.363%
Test Epoch [ 80/200]Batch [  0/ 79] Loss: 0.985 Acc 64.844%
Train Epoch [ 81/200]Batch [  0/391] Loss: 1.052 Acc 65.625%
Train Epoch [ 81/200]Batch [100/391] Loss: 1.120 Acc 61.781%
Train Epoch [ 81/200]Batch [200/391] Loss: 1.118 Acc 61.738%
Train Epoch [ 81/200]Batch [300/391] Loss: 1.117 Acc 61.625%
Test Epoch [ 81/200]Batch [  0/ 79] Loss: 0.953 Acc 66.406%
Saving..
Train Epoch [ 82/200]Batch [  0/391] Loss: 1.066 Acc 62.500%
Train Epoch [ 82/200]Batch [100/391] Loss: 1.119 Acc 61.951%
Train Epoch [ 82/200]Batch [200/391] Loss: 1.119 Acc 61.559%
Train Epoch [ 82/200]Batch [300/391] Loss: 1.117 Acc 61.599%
Test Epoch [ 82/200]Batch [  0/ 79] Loss: 0.967 Acc 66.406%
Train Epoch [ 83/20

Test Epoch [106/200]Batch [  0/ 79] Loss: 0.948 Acc 70.312%
Train Epoch [107/200]Batch [  0/391] Loss: 1.118 Acc 64.844%
Train Epoch [107/200]Batch [100/391] Loss: 1.093 Acc 62.028%
Train Epoch [107/200]Batch [200/391] Loss: 1.089 Acc 62.574%
Train Epoch [107/200]Batch [300/391] Loss: 1.089 Acc 62.664%
Test Epoch [107/200]Batch [  0/ 79] Loss: 0.921 Acc 70.312%
Saving..
Train Epoch [108/200]Batch [  0/391] Loss: 1.224 Acc 57.031%
Train Epoch [108/200]Batch [100/391] Loss: 1.071 Acc 63.041%
Train Epoch [108/200]Batch [200/391] Loss: 1.081 Acc 62.683%
Train Epoch [108/200]Batch [300/391] Loss: 1.081 Acc 62.775%
Test Epoch [108/200]Batch [  0/ 79] Loss: 0.930 Acc 67.188%
Train Epoch [109/200]Batch [  0/391] Loss: 1.096 Acc 63.281%
Train Epoch [109/200]Batch [100/391] Loss: 1.087 Acc 62.330%
Train Epoch [109/200]Batch [200/391] Loss: 1.077 Acc 62.970%
Train Epoch [109/200]Batch [300/391] Loss: 1.076 Acc 62.965%
Test Epoch [109/200]Batch [  0/ 79] Loss: 0.911 Acc 70.312%
Train Epoch [110/20

...

Train Epoch [ 54/200]Batch [  0/391] Loss: 1.123 Acc 63.281%
Train Epoch [ 54/200]Batch [100/391] Loss: 1.067 Acc 63.923%
Train Epoch [ 54/200]Batch [200/391] Loss: 1.075 Acc 63.775%
Train Epoch [ 54/200]Batch [300/391] Loss: 1.076 Acc 63.629%
Test Epoch [ 54/200]Batch [  0/ 79] Loss: 0.889 Acc 73.438%
Train Epoch [ 55/200]Batch [  0/391] Loss: 1.152 Acc 60.938%
Train Epoch [ 55/200]Batch [100/391] Loss: 1.060 Acc 64.310%
Train Epoch [ 55/200]Batch [200/391] Loss: 1.069 Acc 63.806%
Train Epoch [ 55/200]Batch [300/391] Loss: 1.071 Acc 63.761%
Test Epoch [ 55/200]Batch [  0/ 79] Loss: 0.882 Acc 73.438%
Saving..
Train Epoch [ 56/200]Batch [  0/391] Loss: 1.102 Acc 60.156%
Train Epoch [ 56/200]Batch [100/391] Loss: 1.076 Acc 63.312%
Train Epoch [ 56/200]Batch [200/391] Loss: 1.082 Acc 63.056%
Train Epoch [ 56/200]Batch [300/391] Loss: 1.075 Acc 63.414%
Test Epoch [ 56/200]Batch [  0/ 79] Loss: 0.877 Acc 74.219%
Train Epoch [ 57/200]Batch [  0/391] Loss: 0.908 Acc 69.531%
Train Epoch [ 57/2

Train Epoch [ 81/200]Batch [  0/391] Loss: 0.952 Acc 64.062%
Train Epoch [ 81/200]Batch [100/391] Loss: 1.067 Acc 63.660%
Train Epoch [ 81/200]Batch [200/391] Loss: 1.050 Acc 64.354%
Train Epoch [ 81/200]Batch [300/391] Loss: 1.044 Acc 64.423%
Test Epoch [ 81/200]Batch [  0/ 79] Loss: 0.858 Acc 75.000%
Saving..
Train Epoch [ 82/200]Batch [  0/391] Loss: 1.062 Acc 60.156%
Train Epoch [ 82/200]Batch [100/391] Loss: 1.028 Acc 64.991%
Train Epoch [ 82/200]Batch [200/391] Loss: 1.035 Acc 64.797%
Train Epoch [ 82/200]Batch [300/391] Loss: 1.037 Acc 64.883%
Test Epoch [ 82/200]Batch [  0/ 79] Loss: 0.858 Acc 76.562%
Train Epoch [ 83/200]Batch [  0/391] Loss: 0.966 Acc 68.750%
Train Epoch [ 83/200]Batch [100/391] Loss: 1.034 Acc 65.022%
Train Epoch [ 83/200]Batch [200/391] Loss: 1.037 Acc 64.859%
Train Epoch [ 83/200]Batch [300/391] Loss: 1.030 Acc 64.974%
Test Epoch [ 83/200]Batch [  0/ 79] Loss: 0.849 Acc 74.219%
Train Epoch [ 84/200]Batch [  0/391] Loss: 0.892 Acc 69.531%
Train Epoch [ 84/2

Test Epoch [107/200]Batch [  0/ 79] Loss: 0.829 Acc 75.000%
Train Epoch [108/200]Batch [  0/391] Loss: 0.969 Acc 64.062%
Train Epoch [108/200]Batch [100/391] Loss: 1.012 Acc 65.973%
Train Epoch [108/200]Batch [200/391] Loss: 1.016 Acc 65.827%
Train Epoch [108/200]Batch [300/391] Loss: 1.012 Acc 65.840%
Test Epoch [108/200]Batch [  0/ 79] Loss: 0.823 Acc 74.219%
Train Epoch [109/200]Batch [  0/391] Loss: 1.081 Acc 65.625%
Train Epoch [109/200]Batch [100/391] Loss: 1.017 Acc 65.323%
Train Epoch [109/200]Batch [200/391] Loss: 1.009 Acc 65.574%
Train Epoch [109/200]Batch [300/391] Loss: 1.010 Acc 65.586%
Test Epoch [109/200]Batch [  0/ 79] Loss: 0.805 Acc 75.000%
Train Epoch [110/200]Batch [  0/391] Loss: 1.140 Acc 59.375%
Train Epoch [110/200]Batch [100/391] Loss: 1.001 Acc 65.695%
Train Epoch [110/200]Batch [200/391] Loss: 1.011 Acc 65.683%
Train Epoch [110/200]Batch [300/391] Loss: 1.011 Acc 65.612%
Test Epoch [110/200]Batch [  0/ 79] Loss: 0.809 Acc 76.562%
Saving..
Train Epoch [111/20

Test Epoch [134/200]Batch [  0/ 79] Loss: 0.828 Acc 73.438%
Train Epoch [135/200]Batch [  0/391] Loss: 0.792 Acc 74.219%
Train Epoch [135/200]Batch [100/391] Loss: 0.990 Acc 66.236%
Train Epoch [135/200]Batch [200/391] Loss: 0.989 Acc 66.445%
Train Epoch [135/200]Batch [300/391] Loss: 0.996 Acc 66.251%
Test Epoch [135/200]Batch [  0/ 79] Loss: 0.814 Acc 74.219%
Train Epoch [136/200]Batch [  0/391] Loss: 1.012 Acc 71.094%
Train Epoch [136/200]Batch [100/391] Loss: 0.987 Acc 66.074%
Train Epoch [136/200]Batch [200/391] Loss: 0.992 Acc 65.905%
Train Epoch [136/200]Batch [300/391] Loss: 0.992 Acc 66.017%
Test Epoch [136/200]Batch [  0/ 79] Loss: 0.809 Acc 76.562%
Train Epoch [137/200]Batch [  0/391] Loss: 0.948 Acc 68.750%
Train Epoch [137/200]Batch [100/391] Loss: 0.997 Acc 66.569%
Train Epoch [137/200]Batch [200/391] Loss: 0.986 Acc 66.892%
Train Epoch [137/200]Batch [300/391] Loss: 0.990 Acc 66.474%
Test Epoch [137/200]Batch [  0/ 79] Loss: 0.821 Acc 75.000%
Train Epoch [138/200]Batch [

Test Epoch [161/200]Batch [  0/ 79] Loss: 0.825 Acc 75.781%
Train Epoch [162/200]Batch [  0/391] Loss: 1.015 Acc 58.594%
Train Epoch [162/200]Batch [100/391] Loss: 0.964 Acc 67.497%
Train Epoch [162/200]Batch [200/391] Loss: 0.972 Acc 67.129%
Train Epoch [162/200]Batch [300/391] Loss: 0.976 Acc 67.027%
Test Epoch [162/200]Batch [  0/ 79] Loss: 0.815 Acc 72.656%
Train Epoch [163/200]Batch [  0/391] Loss: 0.860 Acc 66.406%
Train Epoch [163/200]Batch [100/391] Loss: 0.995 Acc 66.190%
Train Epoch [163/200]Batch [200/391] Loss: 0.988 Acc 66.363%
Train Epoch [163/200]Batch [300/391] Loss: 0.981 Acc 66.502%
Test Epoch [163/200]Batch [  0/ 79] Loss: 0.811 Acc 74.219%
Train Epoch [164/200]Batch [  0/391] Loss: 0.821 Acc 73.438%
Train Epoch [164/200]Batch [100/391] Loss: 1.002 Acc 66.476%
Train Epoch [164/200]Batch [200/391] Loss: 0.985 Acc 66.678%
Train Epoch [164/200]Batch [300/391] Loss: 0.983 Acc 66.432%
Test Epoch [164/200]Batch [  0/ 79] Loss: 0.794 Acc 73.438%
Train Epoch [165/200]Batch [

In [ ]:
k = 0
plt.figure(figsize=(18, 15))
for i, lr in enumerate(tqdm_notebook(LRS)):    
    for j, norm in enumerate(tqdm_notebook([None,'BN', 'SN', 'MSN'])):
        print(k)
        train_loss_log = np.load(SAVE_PATH+"DenseNet_Train_loss_{}_{}.npy".format(norm,lr) )  
        test_loss_log = np.load(SAVE_PATH+"DenseNet_Test_loss_{}_{}.npy".format(norm,lr))    
        train_acc_log = np.load(SAVE_PATH+"DenseNet_Train_Acc_{}_{}.npy".format(norm,lr))    
        test_acc_log= np.load(SAVE_PATH+"DenseNet_Test_Acc_{}_{}.npy".format(norm,lr))   
        
        ax = plt.subplot(3,4,k+1)
        plt.plot(train_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Loss', fontsize=18)     
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+2)
        plt.plot(test_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Loss', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+3)
        plt.plot(train_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Accuracy', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
                
        ax = plt.subplot(3,4,k+4)
        plt.plot(test_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Accuracy', fontsize=18)        
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
    k+= 4
plt.tight_layout()
plt.savefig(SAVE_PATH+'Act_Norm_Results.pdf', dpi=400)

In [ ]:
k = 0
plt.figure(figsize=(18, 15))
for i, lr in enumerate(tqdm_notebook(LRS)):    
    for j, norm in enumerate(tqdm_notebook(['WN', 'SN', 'MSN'])):
        print(k)
        train_loss_log = np.load(SAVE_PATH+"DenseNet_Train_loss_{}_{}.npy".format(norm,lr) )  
        test_loss_log = np.load(SAVE_PATH+"DenseNet_Test_loss_{}_{}.npy".format(norm,lr))    
        train_acc_log = np.load(SAVE_PATH+"DenseNet_Train_Acc_{}_{}.npy".format(norm,lr))    
        test_acc_log= np.load(SAVE_PATH+"DenseNet_Test_Acc_{}_{}.npy".format(norm,lr))   
        
        ax = plt.subplot(3,4,k+1)
        plt.plot(train_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Loss', fontsize=18)     
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+2)
        plt.plot(test_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Loss', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+3)
        plt.plot(train_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Accuracy', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
                
        ax = plt.subplot(3,4,k+4)
        plt.plot(test_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Accuracy', fontsize=18)        
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
    k+= 4
plt.tight_layout()
plt.savefig(SAVE_PATH+'Weight_reparam_Results.pdf', dpi=400)

In [ ]:
k = 0
plt.figure(figsize=(18, 15))
for i, lr in enumerate(tqdm_notebook(LRS)):    
    for j, norm in enumerate(tqdm_notebook(['MSNTReLU', 'WNTReLU'])):
        print(k)
        train_loss_log = np.load(SAVE_PATH+"DenseNet_Train_loss_{}_{}.npy".format(norm,lr) )  
        test_loss_log = np.load(SAVE_PATH+"DenseNet_Test_loss_{}_{}.npy".format(norm,lr))    
        train_acc_log = np.load(SAVE_PATH+"DenseNet_Train_Acc_{}_{}.npy".format(norm,lr))    
        test_acc_log= np.load(SAVE_PATH+"DenseNet_Test_Acc_{}_{}.npy".format(norm,lr))   
        
        ax = plt.subplot(3,4,k+1)
        plt.plot(train_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Loss', fontsize=18)     
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+2)
        plt.plot(test_loss_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Loss', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);

        ax = plt.subplot(3,4,k+3)
        plt.plot(train_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Train Accuracy', fontsize=18) 
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
                
        ax = plt.subplot(3,4,k+4)
        plt.plot(test_acc_log, lw=2.5, label=str(norm))
        plt.xlabel('Iterations', fontsize=18)
        plt.ylabel('Test Accuracy', fontsize=18)        
        plt.legend(fontsize=18)
        plt.grid(True)
        plt.title("Learning Rate ={}".format(lr), fontsize=20);
    k+= 4
plt.tight_layout()
plt.savefig(SAVE_PATH+'Weight_reparam_act_Results.pdf', dpi=400)